In [1]:
import torch

In [2]:
from datasets import load_dataset

ds = load_dataset("maharshipandya/spotify-tracks-dataset")

Generating train split:   0%|          | 0/114000 [00:00<?, ? examples/s]

In [22]:
import pandas as pd

df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")
df.dropna(inplace=True)

In [28]:
# only keep the columns we need - album_name, track_name, artists, popularity, track_genre
df = df[["album_name", "track_name", "artists", "popularity", "track_genre"]]

# only keep the top 10 genres
top_genres = df["track_genre"].value_counts().head(5).index
df = df[df["track_genre"].isin(top_genres)]

unique_genres = df["track_genre"].unique()
num_genres = len(unique_genres)
unique_genres, num_genres

# randomly sample 40 tracks from each genre
df = df.groupby("track_genre").apply(lambda x: x.sample(40)).reset_index(drop=True)


# create a new column which contains the concatenation of the album_name, track_name and artists in format "track name: {track_name} | album name: {album_name} | artists: {artists}"
df["track_info"] = df.apply(lambda x: f"track name: {x['track_name']} | album name: {x['album_name']} | artists: {x['artists']}", axis=1)

df.head()

,album_name,track_name,artists,popularity,track_genre,track_info
0,Don't Wanna Leave You,Don't Wanna Leave You,Ben Woodward,38,acoustic,track name: Don't Wanna Leave You | album name...
1,Binibini,Binibini,Zack Tabudlo,67,acoustic,track name: Binibini | album name: Binibini | ...
2,21 Guns,21 Guns,Bailey Jehl,47,acoustic,track name: 21 Guns | album name: 21 Guns | ar...
3,Howlin' Wolf,Spoonful,Howlin' Wolf,58,acoustic,track name: Spoonful | album name: Howlin' Wol...
4,"Guitar Covers, Vol. 2","He's A Pirate (From ""Pirates of the Caribbean"")",Beyond The Guitar,30,acoustic,"track name: He's A Pirate (From ""Pirates of th..."
